In [ ]:
pip install missingno

First we import the libraries

In [ ]:
import pandas as pd ## data preprocessing
import numpy as np
import missingno as msno
%matplotlib inline

import matplotlib.pyplot as plt ## graphical analysis
import seaborn as sns
import matplotlib
import re # for regular expressions
import pandas as pd 
 
import string
import nltk # for text manipulation
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from scipy import stats 
from sklearn import metrics 
from sklearn.metrics import mean_squared_error,mean_absolute_error, make_scorer,classification_report,confusion_matrix,accuracy_score,roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb
import warnings 
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
corona_df=pd.read_csv('Corona_NLP_train.csv',  encoding='latin-1')

In [ ]:
corona_df.head()

In [ ]:
corona_df.info()

In [ ]:
corona_df.shape

In [ ]:
corona_df.TweetAt.value_counts()

In [ ]:
corona_df['Location'].value_counts()

In [ ]:
corona_df['Sentiment'].value_counts()

Visualising missing values

In [ ]:
# Gives positional information of the missing values
msno.matrix(corona_df)

Looking at the missing values we noticed that there was some kind of missingness in the location so we decided to clean the data first

Now in the subsequent steps we will be cleaning the data according to natural language processing we have to remove the unwanted words because machine has to understand the sentiment and we used nltk library which does stemming ,tokenisation and lemmitisation

In [ ]:
# in order to remove@user 
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i,'',input_txt)
    return input_txt

In [ ]:
# created a column after removing @ user
corona_df['Tweet'] = np.vectorize(remove_pattern)(corona_df['OriginalTweet'], '@[\w]*')

import re has been used for regural expressions

In [ ]:
import re
corona_df['Tweet'] = corona_df['Tweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

In [ ]:
#2. removed stop words,punctuations
corona_df['Tweet'] = corona_df['Tweet'].str.replace('[^a-zA-Z#]+',' ')

In [ ]:
# 3.removed short words
corona_df['Tweet'] = corona_df['Tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2]))

In [ ]:
# tokenisation
tokenized_tweet = corona_df['Tweet'].apply(lambda x: x.split())

Stemming-Process of reducing infected words to their words stem

In [ ]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

# apply stemmer for tokenized_tweet
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])

In [ ]:
# appending/joining two features
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

New Cleaned dataset after feature engineering

In [ ]:
covid19 = corona_df[['Tweet','Sentiment']]

In [ ]:
covid19.head(10)

,Tweet,Sentiment
0,,Neutral
1,advice Talk your neighbours family exchange ph...,Positive
2,Coronavirus Australia Woolworths give elderly ...,Positive
3,food stock not the only one which empty PLEASE...,Positive
4,ready supermarket during the #COVID outbreak N...,Extremely Negative
5,news the region first confirmed COVID case cam...,Positive
6,Cashier grocery store was sharing his insights...,Positive
7,Was the supermarket today Didn buy toilet pape...,Neutral
8,Due COVID our retail store and classroom Atlan...,Positive
9,For corona prevention should stop buy things w...,Negative


Removing stopwords

In [ ]:
covid19["Tweet"] = covid19["Tweet"].str.lower()#.str.split()

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
covid19['Tweet'].apply(lambda x: [item for item in x if item not in stop])

Dividing the data into 80 and 20

In [ ]:
from sklearn.model_selection import train_test_split

train,valid = train_test_split(covid19,test_size = 0.2,random_state=0,stratify = covid19.Sentiment.values)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stop = list(stopwords.words('english'))
vectorizer = CountVectorizer(decode_error = 'replace',stop_words = stop)

X_train = vectorizer.fit_transform(train.Tweet.values)
X_valid = vectorizer.transform(valid.Tweet.values)

y_train = train.Sentiment.values
y_valid = valid.Sentiment.values


Random Forest Classifier is maximum used in nlp becuase it gives much accurate results as the answer is based on the no of trees

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

rf.fit(X_train,y_train)

In [ ]:
## Importing library to check accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [ ]:
rf_predict = rf.predict(X_valid)
rf_accu = accuracy_score(y_valid,rf_predict)
print("Training accuracy Score    : ",rf.score(X_train,y_train))
print(classification_report(rf_predict,y_valid))